# **Análisis de Boletín Oficial con Machine Learning**

Descripción del pipeline:

1. Toma un Boletín Oficial (PDF).
2. Extrae fragmentos candidatos mediante la búsqueda de términoc clave.
3. Usa un modelo TF-IDF + SVM ya entrenado para decidir si el fragmento indica que se trata de una norma relevante para el ejercicio profesional de la arquitectura y el urbanismo.
4. Muestra los resultados de forma legible.
5. Permite ir incorporando nuevas normas etiquetadas para re entrenar el modelo con el tiempo.

## **Celda 1 - Montar Drive y definir rutas**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

BASE = "/content/drive/MyDrive/IA/Proyectos/Análisis Boletín Oficial/boletin-ml"

# PDFs nuevos (los BO que querés revisar hoy)
DIR_PDF = BASE + "/data/raw/boletines_diarios"

# Modelo SVM+TFIDF guardado desde el baseline
MODEL_PATH = BASE + "/models/demo_svm/svm_tfidf_pipeline.joblib"

# Carpeta de salida de resultados
DIR_OUT = BASE + "/inference"
import os
os.makedirs(DIR_OUT, exist_ok=True)

print('Directorios cargados ✅')

Mounted at /content/drive
Directorios cargados ✅


## **Celda 2 – Imports y carga del modelo**

In [2]:
!pip install pypdf2 > /dev/null

import os
import re
import pandas as pd
import numpy as np
import PyPDF2
from joblib import load
from tqdm.notebook import tqdm

# Cargar el pipeline SVM+TF-IDF entrenado
svm_pipeline = load(MODEL_PATH)
print("Modelo cargado ✅:", type(svm_pipeline))

print('Listo ✅')

Modelo cargado ✅: <class 'sklearn.pipeline.Pipeline'>
Listo ✅


## **Celda 3 - Listas de palabras clave**



In [3]:
VERBOS_ACCION = [
    r"\b[Mm]odifica\b", r"\b[Mm]odifícase\b", r"\b[Mm]odificar\b",
    r"\b[Dd]eroga\b", r"\b[Dd]erogar\b", r"\b[Dd]erógase\b",
    r"\b[Aa]prueba\b", r"\b[Aa]probar\b", r"\b[Aa]pruébese\b",
    r"\b[Dd]eja sin efecto\b", r"\b[Dd]ejar sin efecto\b", r"\b[Dd]éjase sin efecto\b",
    r"\b[Ss]ustituye\b", r"\b[Ss]ustituir\b", r"\b[Ss]ustitúyese\b",
    r"\b[Ee]stablece\b", r"\b[Ee]stablecer\b", r"\b[Ee]stablécese\b",
    r"\b[Ff]ija\b", r"\b[Ff]ijar\b", r"\b[Ff]íjese\b",
    r"\b[Dd]etermina\b", r"\b[Dd]etermínase\b", r"\b[Dd]eterminar\b",
    r"\b[Rr]eglamenta\b", r"\b[Rr]eglaméntese\b", r"\b[Rr]eglamentación\b",
    r"\b[Pp]rorroga\b", r"\b[Pp]rorrógase\b", r"\b[Pp]rorrogar\b"
]

KEYWORDS = [
    # Construcción
    "código de edificación",
    "reglamentos técnicos",
    "reglamento técnico",
    "planos de mensura",
    "planos de obra",
    "planos de instalaciones",
    "obras en contravención",
    "accesibilidad",
    "conservación de fachadas",

    # Digestos normativos
    "compendio normativo",
    "digesto",

    # Escuelas seguras
    "ueresgp",
    "régimen de escuelas seguras de gestión privada",

    # Espacio público
    "uso del espacio público",
    "publicidad exterior",

    # Fiscal y tarifario
    "ley tarifaria",
    "código fiscal",
    "unidad tarifaria",
    "derecho para el desarrollo urbano y el hábitat sustentable",
    "derecho de construcción sustentable",
    "derecho de construcción",

    # Habilitaciones
    "código de habilitaciones",
    "autorización de actividad económica",
    "autorización de actividades económicas",
    "autorizaciones de actividades económicas",
    "habilitación",
    "habilitación económica",
    "ley marco de regulación de actividades económicas",

    # Impacto ambiental
    "impacto ambiental",
    "certificado de aptitud ambiental",

    # Sistema de Autoprotección
    "sistema de autoprotección",
    "sistemas de autoprotección",

    # Urbanismo
    "código urbanístico",
    "catastro",
    "área céntrica",
    "reurbanización",
    "zonificación",
    "planeamiento urbano",
    "normas urbanísticas",
    "convenios urbanísticos",
    "convenio urbanístico"
]

PATRONES_NORMAR = [
    # Ordenanzas
    r"[Oo]rdenanza(?: [Nn]°?)? ?3\.?442",
    r"[Oo]rdenanza(?: [Nn]°?)? ?3\.?421",

    # Disposiciones
    r"[Dd]isposici[oó]n(?: [Nn]°?)? ?3\.?500(?:[-/]?GCABA)?[-/]?DGOEP[-/]?16",
    r"[Dd]isposici[oó]n(?: [Nn]°?)? ?89(?:[-/]?GCABA)?[-/]?DGROC[-/]?24",
    r"[Dd]isposici[oó]n(?: [Nn]°?)? ?526(?:[-/]?GCABA)?[-/]?DGFYCO[-/]?24",
    r"[Dd]isposici[oó]n(?: [Nn]°?)? ?331(?:[-/]?GCABA)?[-/]?DGDCIV[-/]?25",

    # Resoluciones
    r"[Rr]esoluci[oó]n(?: [Nn]°?)? ?345(?:[-/]?GCABA)?[-/]?AGC[-/]?21",
    r"[Rr]esoluci[oó]n(?: [Nn]°?)? ?275(?:[-/]?GCABA)?[-/]?APRA[-/]?23",
    r"[Rr]esoluci[oó]n(?: [Nn]°?)? ?160(?:[-/]?GCABA)?[-/]?SSHA[-/]?24",
    r"[Rr]esoluci[oó]n(?: [Nn]°?)? ?188(?:[-/]?GCABA)?[-/]?SSGU[-/]?24",
    r"[Rr]esoluci[oó]n(?: [Nn]°?)? ?1(?:[-/]?GCABA)?[-/]?MEPHUGC[-/]?25",
    r"[Rr]esoluci[oó]n(?: [Nn]°?)? ?96(?:[-/]?GCABA)?[-/]?AGC[-/]?25",
    r"[Rr]esoluci[oó]n(?: [Nn]°?)? ?315(?:[-/]?GCABA)?[-/]?APRA[-/]?25",

    # Decretos
    r"[Dd]ecreto(?: [Nn]°?)? ?51/18",
    r"[Dd]ecreto(?: [Nn]°?)? ?85/19",
    r"[Dd]ecreto(?: [Nn]°?)? ?86/19",
    r"[Dd]ecreto(?: [Nn]°?)? ?87/19",
    r"[Dd]ecreto(?: [Nn]°?)? ?99/19",
    r"[Dd]ecreto(?: [Nn]°?)? ?105/19",
    r"[Dd]ecreto(?: [Nn]°?)? ?475/20",
    r"[Dd]ecreto(?: [Nn]°?)? ?116/25",
    r"[Dd]ecreto(?: [Nn]°?)? ?129/25",
    r"[Dd]ecreto(?: [Nn]°?)? ?164/25",
    r"[Dd]ecreto(?: [Nn]°?)? ?189/25",

    # Leyes
    r"[Ll]ey(?: [Nn]°?)? ?123",
    r"[Ll]ey(?: [Nn]°?)? ?2\.?189",
    r"[Ll]ey(?: [Nn]°?)? ?2\.?936",
    r"[Ll]ey(?: [Nn]°?)? ?5\.?920",
    r"[Ll]ey(?: [Nn]°?)? ?6\.099",
    r"[Ll]ey(?: [Nn]°?)? ?6\.100",
    r"[Ll]ey(?: [Nn]°?)? ?6\.101",
    r"[Ll]ey(?: [Nn]°?)? ?6\.437",
    r"[Ll]ey(?: [Nn]°?)? ?6\.438",
    r"[Ll]ey(?: [Nn]°?)? ?6\.508",
    r"[Ll]ey(?: [Nn]°?)? ?6\.509",
    r"[Ll]ey(?: [Nn]°?)? ?6\.769"
    r"[Ll]ey(?: [Nn]°?)? ?6\.776",
    r"[Ll]ey(?: [Nn]°?)? ?6\.779",
    r"[Ll]ey(?: [Nn]°?)? ?6\.806",
]

print('Listas cargadas ✅')

Listas cargadas ✅


## **Celda 4 – Umbral del SVM**

In [4]:
THRESHOLD_SVM = 0.018
print("Umbral SVM (decision_function):", THRESHOLD_SVM)

Umbral SVM (decision_function): 0.018


## **Celda 5 – Lectura de PDF, segmentación y búsqueda de candidatos**

In [8]:
def read_pdf_text(path):
    """Lee todo el texto de un PDF en un único string."""
    reader = PyPDF2.PdfReader(path)
    texts = []
    for page in reader.pages:
        t = page.extract_text()
        if t:
            texts.append(t)
    return "\n".join(texts)

def sent_segment(text):
    """Segmenta en oraciones (podés reemplazar por la que uses en Dataset Builder)."""
    sents = re.split(r'(?<=[\.\?\!])\s+(?=[A-ZÁÉÍÓÚÑ0-9])', text)
    sents = [re.sub(r"\s+", " ", s).strip() for s in sents if s and s.strip()]
    return sents

def any_match(patterns, text, use_regex=True):
    t = text.lower()
    if use_regex:
        return any(re.search(p, t) for p in patterns)
    else:
        return any(p.lower() in t for p in patterns)

def deduplicate_contexts(df):
    """Elimina duplicados exactos y contextos que son subcadenas de otros en la misma página."""
    if df.empty:
        return df

    df = df.copy()
    df["contexto_norm"] = df["contexto"].str.replace(r"\s+", " ", regex=True).str.strip()

    # 1) quitar duplicados exactos por pdf + página + texto normalizado
    df = df.drop_duplicates(subset=["origen_pdf", "page", "contexto_norm"])

    # 2) eliminar subcadenas dentro de la misma página (dejar el contexto más largo)
    def drop_substrings(group):
        ctx = group["contexto_norm"].tolist()
        keep = [True] * len(ctx)
        for i, ci in enumerate(ctx):
            for j, cj in enumerate(ctx):
                if i != j and len(ci) < len(cj) and ci in cj:
                    keep[i] = False
                    break
        return group.loc[keep]

    df = df.groupby(["origen_pdf", "page"], group_keys=False).apply(drop_substrings)

    df = df.drop(columns=["contexto_norm"])
    return df.reset_index(drop=True)

def extract_candidates_from_pdf(pdf_path):
    """
    Lee el PDF página por página, segmenta en oraciones y arma fragmentos candidatos
    usando las reglas (KEYWORDS, PATRONES_NORMAR, VERBOS_ACCION).
    Incluye número de página y elimina fragmentos duplicados/superpuestos.
    """
    reader = PyPDF2.PdfReader(pdf_path)
    rows = []

    for page_idx, page in enumerate(tqdm(reader.pages,
                                         desc="Buscando candidatos",
                                         unit="página")):
        text = page.extract_text() or ""
        sents = sent_segment(text)

        for i, sent in enumerate(sents):
            left  = sents[i-1] if i-1 >= 0 else ""
            right = sents[i+1] if i+1 < len(sents) else ""
            contexto = " ".join([left, sent, right]).strip()

            has_keyword = any_match(KEYWORDS, contexto, use_regex=False)
            has_normar  = any_match(PATRONES_NORMAR, contexto, use_regex=True)
            has_verbo   = any_match(VERBOS_ACCION, contexto, use_regex=True)

            cond = has_verbo and (has_keyword or has_normar)

            if cond:
              flags = []
              if has_verbo:   flags.append("VERBO")
              if has_keyword: flags.append("KEYWORD")
              if has_normar:  flags.append("NORMAR")

              rows.append({
                  "origen_pdf": os.path.basename(pdf_path),
                  "page":       page_idx + 1,
                  "sent_idx":   i,
                  "contexto":   contexto,
                  "origen_flags": ";".join(flags)
              })

    df = pd.DataFrame(rows)
    df = deduplicate_contexts(df)
    return df

print('Funciones cargadas ✅')

Funciones cargadas ✅


## **Celda 6 - Clasificación de resultados con SVM**

In [9]:
def classify_candidates(df_cand, threshold=THRESHOLD_SVM, batch_size=256):
    """
    Clasifica los candidatos con SVM usando decision_function.
    Añade:
      - score_svm
      - pred_pertinente (0/1 según umbral)
    """
    if df_cand.empty:
        return df_cand.assign(score_svm=[], pred_pertinente=[])

    texts = df_cand["contexto"].astype(str).tolist()
    scores = []

    for start in tqdm(range(0, len(texts), batch_size),
                      desc="Clasificando con SVM",
                      unit="lote"):
        batch = texts[start:start+batch_size]
        s = svm_pipeline.decision_function(batch)
        scores.extend(list(s))

    scores = np.array(scores)
    df_out = df_cand.copy()
    df_out["score_svm"] = scores
    df_out["pred_pertinente"] = (scores >= threshold).astype(int)
    return df_out

print('Funciones cargadas ✅')

Funciones cargadas ✅


## **Celda 7 - Ejecución sobre un Boletín y guardado de resultados**

In [10]:
def get_latest_pdf(dir_path=DIR_PDF):
    """Devuelve la ruta al PDF más reciente dentro de DIR_PDF."""
    pdfs = [os.path.join(dir_path, f)
            for f in os.listdir(dir_path)
            if f.lower().endswith(".pdf")]
    if not pdfs:
        raise FileNotFoundError(f"No hay PDFs en {dir_path}")
    latest = max(pdfs, key=os.path.getmtime)
    return latest

def run_latest_pdf(threshold=THRESHOLD_SVM, top_n=50):
    pdf_path = get_latest_pdf(DIR_PDF)
    print("Último PDF encontrado:", os.path.basename(pdf_path))

    df_cand = extract_candidates_from_pdf(pdf_path)
    print("Candidatos después de deduplicar:", len(df_cand))

    if df_cand.empty:
        return df_cand, pd.DataFrame()

    df_pred = classify_candidates(df_cand, threshold=threshold)

    n_pos = int(df_pred["pred_pertinente"].sum())
    print(f"Marcados como pertinentes (umbral={threshold}): {n_pos}")

    df_sorted = df_pred.sort_values("score_svm", ascending=False)

    from IPython.display import display
    display(df_sorted.head(top_n)[[
        "origen_pdf", "page", "origen_flags",
        "score_svm", "pred_pertinente", "contexto"
    ]])

    out_name = os.path.splitext(os.path.basename(pdf_path))[0] + "_svm_preds.csv"
    out_path = os.path.join(DIR_OUT, out_name)
    df_sorted.to_csv(out_path, index=False, encoding="utf-8-sig", sep=";")
    print("Resultados guardados en:", out_path)

    return df_cand, df_sorted

df_cand, df_pred = run_latest_pdf()

Último PDF encontrado: 20251104.pdf


Buscando candidatos:   0%|          | 0/748 [00:00<?, ?página/s]

Candidatos después de deduplicar: 53


/tmp/ipython-input-3460215793.py:46: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby(["origen_pdf", "page"], group_keys=False).apply(drop_substrings)


Clasificando con SVM:   0%|          | 0/1 [00:00<?, ?lote/s]

Marcados como pertinentes (umbral=0.018): 19


,origen_pdf,page,origen_flags,score_svm,pred_pertinente,contexto
10,20251104.pdf,45,VERBO;KEYWORD;NORMAR,1.175423,1,"36, 37 y 42 de la Ley N° 123. Artículo 4°. - A..."
20,20251104.pdf,46,VERBO;KEYWORD,1.073834,1,"Artículo 17°.- Aprobar el ""Reglamento de la Co..."
12,20251104.pdf,45,VERBO;KEYWORD,1.064806,1,"Artículo 5°. - Aprobar el ""Procedimiento para ..."
11,20251104.pdf,45,VERBO;KEYWORD,1.052516,1,Artículo 4°. - Aprobar el “Cuadro de Categoriz...
14,20251104.pdf,45,VERBO;KEYWORD,0.978278,1,Artículo 7°.- Aprobar las condiciones de inscr...
9,20251104.pdf,45,VERBO;NORMAR,0.976496,1,"La falsedad, omisión u ocultamiento de la info..."
13,20251104.pdf,45,VERBO;KEYWORD,0.879782,1,"Artículo 6°. - Aprobar el ""Procedimiento de Re..."
15,20251104.pdf,45,VERBO;KEYWORD,0.867980,1,"Artículo 8°. - Aprobar el ""Formulario de Categ..."
16,20251104.pdf,45,VERBO;KEYWORD,0.734519,1,"Artículo 9°. - Aprobar el ""Formulario de categ..."
8,20251104.pdf,45,VERBO;KEYWORD;NORMAR,0.675511,1,Artículo 3°. - Establecer que las presentacion...


Resultados guardados en: /content/drive/MyDrive/IA/Proyectos/Análisis Boletín Oficial/boletin-ml/inference/20251104_svm_preds.csv
